In [21]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os
import math
import re
import time


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [3]:
def get_cards_list(api_key, updated_at=None, nm_id=None, limit=100):
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    
    cards_list_url = "https://content-api.wildberries.ru/content/v2/get/cards/list"
    
    # Базовый запрос
    body = {
        "settings": {
            "cursor": {
                "limit": limit
            },
            "filter": {
                "withPhoto": 1,
                "objectIDs": [3979]
            }
        }
    }
    
    # Добавляем параметры, если они переданы
    if updated_at:
        body["settings"]["cursor"]["updatedAt"] = updated_at
    if nm_id:
        body["settings"]["cursor"]["nmID"] = nm_id
    
    try:
        response = requests.post(cards_list_url, headers=headers, json=body)
        response.raise_for_status()  # Выбросит исключение, если статус ответа не 200
        return response.json()
    except requests.RequestException as e:
        print(f"Ошибка при запросе списка карточек: {e}")
        return None


In [8]:

vendor_code_pattern = re.compile(r"^box_\d+_\d+$")
all_cards = []
updated_at = None
nm_id = None

while True:
    response = get_cards_list(WB_TOKEN, updated_at=updated_at, nm_id=nm_id)

    if not response or "cards" not in response:
        print("Запрос завершён или нет данных.")
        break

    cards = response["cards"]
    if not cards:
        print("Больше нет карточек.")
        break
    
    filtered_cards = [card for card in cards if "vendorCode" in card and vendor_code_pattern.match(card["vendorCode"])]
    all_cards.extend(filtered_cards)

    # Обновляем параметры для следующего запроса
    cursor = response.get("cursor", {})
    updated_at = cursor.get("updatedAt")
    nm_id = cursor.get("nmID")

    # Если нет курсора, значит, это последняя страница
    if not updated_at or not nm_id:
        break

    print(f"Загружено {len(all_cards)} карточек, продолжаем...")



Загружено 100 карточек, продолжаем...
Загружено 200 карточек, продолжаем...
Загружено 270 карточек, продолжаем...
Больше нет карточек.


In [10]:
all_cards

[{'nmID': 323158312,
  'imtID': 304586570,
  'nmUUID': '0194bc4c-a303-7b88-96d4-5ecda5a6a204',
  'subjectID': 3979,
  'subjectName': 'Коробки картонные',
  'vendorCode': 'box_9603052_10',
  'brand': '',
  'title': 'Коробка крафт кондитерская, беленая, 215*150*60 мм 10 шт.',
  'description': 'Коробка крафт кондитерская, беленая, длиной 215 мм, шириной 150 мм и высотой 60 мм. Изготовлена из крафт картона. Полезный объём — 1.94 л.\nОбласть применения:\nПредназначена для упаковки и транспортировки кондитерских изделий — пирожных, чизкейков, печенья, кексов, вафель. Изготовлена из картона светлого оттенка, который придаёт коробке аккуратный вид. Закрывается при помощи крышки — удлиннённой боковой стенки.\nПреимущества:\nЗакрывается крышкой-стенкой\nВозможность маркировки картонной поверхности.\nЗащищает продукты от влаги и солнечных лучей.\nНизкая стоимость.\nИзделия из крафт картона стоит хранить в сухом и чистом помещении при температуре от 0 до 30 ℃ и относительной влажности (65∓15 %) на

In [16]:
vendor_code_pattern = re.compile(r"^box_(\d+)_(\d+)$")  # Корректируем регулярку
filtered_cards = []
for card in all_cards:
    match = vendor_code_pattern.match(card['vendorCode'])
    if match:
        filtered_cards.append(card)

# Группируем карточки по числовому префиксу из vendorCode (box_XXXXXXX)
grouped_by_prefix = {}
for card in filtered_cards:
    match = vendor_code_pattern.match(card['vendorCode'])
    if match:
        prefix = match.group(1)  # Извлекаем число XXXX из box_XXXX_YY
        if prefix not in grouped_by_prefix:
            grouped_by_prefix[prefix] = []
        grouped_by_prefix[prefix].append(card)

In [17]:
final_groups = []
for prefix, cards in grouped_by_prefix.items():
    sorted_cards = sorted(cards, key=lambda x: x['nmID'])  # Сортируем по nmID
    for i in range(0, len(sorted_cards), 3):  # Берем по 3 карточки за раз
        batch = sorted_cards[i:i+3]
        if len(batch) > 1:  # Только если в группе больше 1 карточки
            final_groups.append(batch)


In [22]:
WB_API_URL = "https://content-api.wildberries.ru/content/v2/cards/moveNm"
for group in final_groups:
    imt_id = group[0]['imtID']  # Первый элемент как targetIMT
    nm_ids = [card['nmID'] for card in group]  # nmID карточек для объединения

    payload = {
        "targetIMT": imt_id,
        "nmIDs": nm_ids
    }
    
    response = requests.post(WB_API_URL, json=payload, headers=headers)
    time.sleep(1)
    if response.status_code == 200:
        print(f"✅ Успешно объединены карточки {nm_ids} под imtID {imt_id}")
    else:
        print(f"❌ Ошибка при объединении {nm_ids}: {response.text}")

✅ Успешно объединены карточки [323158308, 323158312, 323158431] под imtID 304586566
✅ Успешно объединены карточки [323158349, 323158484, 323158651] под imtID 304586606
✅ Успешно объединены карточки [323158299, 323158507, 323158712] под imtID 304586556
✅ Успешно объединены карточки [323158265, 323158382, 323158753] под imtID 304586517
✅ Успешно объединены карточки [323158305, 323158476, 323158604] под imtID 304586563
✅ Успешно объединены карточки [323158310, 323158354, 323158684] под imtID 304586568
✅ Успешно объединены карточки [323158332, 323158504, 323158562] под imtID 304586590
✅ Успешно объединены карточки [323158319, 323158379, 323158766] под imtID 304586577
✅ Успешно объединены карточки [323158274, 323158303, 323158719] под imtID 304586526
✅ Успешно объединены карточки [323158271, 323158699, 323158760] под imtID 304586524
✅ Успешно объединены карточки [323158297, 323158301, 323158512] под imtID 304586554
✅ Успешно объединены карточки [323158295, 323158461, 323158528] под imtID 30